The current problem we're facing is

> The simulations run too slow. An average iteration in `Net` below for a classical layer takes ~10ns whereas for a quantum layer it takes ~0.5s, that too parallelized to (8/16 threads)

This is most likely since most of Qiskit is written in Python and not in C-family/Fortran top to bottom. In fact the only C layer in all of Qiskit is the `Aer` package which is why it takes 0.5s and not something like 15s.

There is no way I know of so far of circumventing this problem. the following are a few considerations:
- Using **TorchQuantum**: TorchQuantum does not support Apple Silicon installations (i am using an M2 laptop). The issue is tracked [here](https://github.com/mit-han-lab/torchquantum/issues/98)
- Use the **GPU**: The current Benchmark for M2 CPU is faster than the Colab GPU
- Using **Apple Silicon** to its full extent: While pytorch is already ready for it, Qiskit is not. The issue is tracked [here](https://github.com/Qiskit/qiskit-aer/issues/1762). With full Apple Silicon support, we can use the M2 to basically as much power as the same order of magnitude as Titan
- Using a **Different Algorithm**: See [QCNN.ipynb](./QCNN.ipynb)
- Using **Runtime Primitives**: The `EstimatorQNN` class actually returns the energy levels measured in various ways. The circuit is not learning when using those/I don't know how to use it (since I can't find a lot of examples online)
- Using `TorchConnector`: There is no speed/learning benefit. Under the hood it uses the same `EstimatorQNN` class

### The situation
We know for a fact this model works because under various configurations we are seeing learning happening. For $(0,1)$ case it is almost a perfect classifier. For $(0,1,3,6)$ case for small training size it is random but as the trainset becomes larger it starts becoming more and more better than random (I was able to reach ~40%, perfectly random is ~25%)

It stands to reason if Qiskit were faster we would see the same thing happening for the full MNIST dataset.

In [8]:
from torch.autograd import Function
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import numpy as np
import torch

from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.primitives import Estimator
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit
from time import time

from utils import gtt

n_train = 100
n_test = 10
qubits = 13
shots = 1024

# train_loader, test_loader = gtt(n_train, [i for i in range(10)])
train_loader, test_loader = gtt(n_train, [0, 1, 2])

In [9]:
# NC4 from I, X, Y, Z
paulis = ["I", "X", "Y", "Z"]
def select(n):
    if n == 0:
        return [""]
    if n == 1:
        return ["I", "X", "Y", "Z"]
    return [a+b for a in select(n-1) for b in paulis]

class QuantumCircuitGen:
    def __init__(self, qubits):
        print("Initializing QuantumCircuitGen with", qubits, "qubits")
        self.backend = Estimator()
        possible = select(5 if qubits > 5 else qubits)
        possible = np.random.choice(possible, qubits)
        if qubits > 5:
            possible = ["I"*(qubits-5)+p for p in possible]
        self.obs = [self.op(p) for p in possible]

        circ = QuantumCircuit(qubits)
        for i in range(qubits):
            circ.h(i)
            t = Parameter('t'+str(i))
            circ.cx(i, (i+1)%qubits)
            circ.rx(t, i)
        circ.measure_all()
        self.circuit = circ

    def op(self, p):
        return SparsePauliOp.from_list([(p, 1)])

    def run(self, inputs): # Runs a circuit
        qc = self.circuit.assign_parameters(inputs)
        result = self.backend.run(
            [qc]*len(self.obs),
            self.obs,
            shots=shots,
        ).result()
        return torch.tensor(result.values)

In [10]:
class HybridFunction(Function):
    def forward(ctx, input, quantum_circuit):
        ctx.shift = np.pi / 2;
        ctx.quantum_circuit = quantum_circuit

        results = [];
        for i in range(len(input)):
            expectation_z = ctx.quantum_circuit.run(input[i].tolist())
            results.append(torch.tensor(np.array([expectation_z])))

        # Save the input and the result for the backward pass
        results = torch.stack(results).squeeze(1)
        ctx.save_for_backward(input, results)

        return results

    def backward(ctx, grad_output):
        input, expectation_z = ctx.saved_tensors  # Load the saved tensors
        input_list = np.array(input.tolist())

        shift_right = input_list + np.ones(input_list.shape) * ctx.shift
        shift_left = input_list - np.ones(input_list.shape) * ctx.shift

        gradients = []
        for i in range(len(input_list)):
            # THIS IS WHY IT DOESNT WORK
            # small changes may lead to large differences in the output
            # like 10000 -> 00000 is a difference of 16 not 1
            expectation_right = ctx.quantum_circuit.run(shift_right[i])
            expectation_left = ctx.quantum_circuit.run(shift_left[i])
            print("R:", expectation_right)
            print("L:", expectation_left)

            gradient = torch.tensor(np.array([expectation_right])) - \
                torch.tensor(np.array([expectation_left]))
            print("G:", gradient)
            gradients.append(gradient)

        gradients = torch.stack(gradients).squeeze(1)
        return gradients * grad_output.float(), None, None

class Hybrid(nn.Module):
    def __init__(self):
        super(Hybrid, self).__init__()
        self.quantum_circuit = QuantumCircuitGen(13)

    def forward(self, input):
        return HybridFunction.apply(input, self.quantum_circuit)

In [11]:
class Net(nn.Module): # the actual neural net as mentioned in the tutorial
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=4)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=4)
        out_conv1 = F.max_pool2d(self.conv1(torch.rand(1,1,28,28)), 2);
        out_conv2 = F.max_pool2d(self.conv2(out_conv1), 2)
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(out_conv2.view(1,-1).shape[1], qubits)
        self.hybrid = Hybrid()
        out_hybrid = self.hybrid(torch.rand(qubits,qubits))
        self.fc2 = nn.Linear(out_hybrid.shape[1], 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x) # We don't relu this to prevent learning, we pass as-is to QC
        x = self.hybrid(x).type(torch.FloatTensor)
        x = self.fc2(x)
        return x;

model = Net();
print(model)

Initializing QuantumCircuitGen with 13 qubits


/var/folders/mt/y75db3512zv_8r60rn4pb82h0000gn/T/ipykernel_68315/3309645063.py:13: DeprecationWarning: Option approximation=False is deprecated as of qiskit-aer 0.13. It will be removed no earlier than 3 months after the release date. Instead, use BackendEstmator from qiskit.primitives.
  self.backend = Estimator()


Net(
  (conv1): Conv2d(1, 6, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(4, 4), stride=(1, 1))
  (dropout): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=256, out_features=13, bias=True)
  (hybrid): Hybrid()
  (fc2): Linear(in_features=13, out_features=10, bias=True)
)


In [12]:
optimizer = optim.Adam(model.parameters(), lr=0.001) # Adam optimizer
loss_func = nn.CrossEntropyLoss() # Cross entropy loss since we're doing classification

epochs = 20 # for now 20
loss_list = [3] # we need to intialize this to something, 3 is arbitrary

model.train() # Set the model to training mode

outputs = []
targets = []
for epoch in range(epochs):
    total_loss = []
    times = []
    for batch_idx, (data, target) in enumerate(train_loader):
        now = time()
        optimizer.zero_grad()
        output = model(data)

        outputs.append(torch.argmax(output))
        targets.append(target)

        loss = loss_func(output, target) # Loss

        loss.backward()
        optimizer.step()

        total_loss.append(loss.item())
        times.append(time() - now)

    print(f"Avg Itr Time: {np.round(np.average(times),1)}s x {len(times)} itrs = {np.round(np.sum(times)/60,1)}min")
    loss_list.append(sum(total_loss)/len(total_loss))

    diff = np.abs(loss_list[-1] - loss_list[-2]) /loss_list[-1];
    if diff <= 0.001: # Early stopping criterial loss diff = 0.1%
        break;

    print(f'Training [{100. * (epoch + 1) / epochs:.0f}%]\tLoss: {loss_list[-1]:.4f}')

R: tensor([-0.0137, -0.0098, -0.0078, -0.0391,  0.0000, -0.0332,  0.0000,  0.0059,
         0.0156,  0.0547, -0.0098,  0.0410, -0.0469], dtype=torch.float64)
L: tensor([ 0.0273,  0.0020,  0.0371, -0.0020,  0.0547,  0.0059, -0.0254, -0.0391,
        -0.0195,  0.0059,  0.0215, -0.0410,  0.0215], dtype=torch.float64)
G: tensor([[-0.0410, -0.0117, -0.0449, -0.0371, -0.0547, -0.0391,  0.0254,  0.0449,
          0.0352,  0.0488, -0.0312,  0.0820, -0.0684]], dtype=torch.float64)
R: tensor([-0.0488, -0.0098, -0.0469, -0.0137,  0.0000,  0.0508, -0.0801, -0.0059,
        -0.0039,  0.0352, -0.0508,  0.0293, -0.0059], dtype=torch.float64)
L: tensor([ 0.0059,  0.0430,  0.0625,  0.0098,  0.0039,  0.0469,  0.0000, -0.0195,
         0.0156,  0.0195,  0.0098, -0.0215, -0.0410], dtype=torch.float64)
G: tensor([[-0.0547, -0.0527, -0.1094, -0.0234, -0.0039,  0.0039, -0.0801,  0.0137,
         -0.0195,  0.0156, -0.0605,  0.0508,  0.0352]], dtype=torch.float64)
R: tensor([-0.0215, -0.0410,  0.0039,  0.0156,

KeyboardInterrupt: 

In [ ]:
model.eval() # Set the model to evaluation mode
with torch.no_grad(): # Don't compute gradients
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader): # Loop over the test set
        output = model(data)

        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

        loss = loss_func(output, target)
        total_loss.append(loss.item())

    print('Performance on test data:\n\tLoss: {:.4f}\n\tAccuracy: {:.1f}%'.format(
        sum(total_loss) / len(total_loss),
        correct / len(test_loader) * 100)
        )

In [ ]:
from matplotlib import pyplot as plt
n_samples_show = 6
count = 0
fig, axes = plt.subplots(nrows=1, ncols=n_samples_show, figsize=(10, 3))

model.eval()
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(test_loader):
        if count == n_samples_show:
            break
        output = model(data)
        print(output)

        pred = output.argmax(dim=1, keepdim=True)

        axes[count].imshow(data[0].numpy().squeeze(), cmap='gray')

        axes[count].set_xticks([])
        axes[count].set_yticks([])
        axes[count].set_title('Predicted {}'.format(pred.item()))

        count += 1

In [ ]:
# For dark mode
from IPython.core.display import HTML
HTML("""
<style>
  html{filter:invert(1)}
  div.prompt{opacity: 0.5;}
  .btn-default{border-color: transparent;}
  #header-container{display:none !important;}
  div.cell.selected, div.cell.selected.jupyter-soft-selected{border-color: transparent;}
</style>
""")